In [1]:
import pandas as pd
import os

from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import *
from sklearn.linear_model import LogisticRegression

In [2]:
path = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/spam-filtering"

filenames = os.listdir(path)
videos = [pd.read_csv(path + '/' + filename, sep=',') for filename in filenames]

dataset = pd.concat(videos)

In [3]:
dataset.sample(frac=1).head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
97,z134d5u5gkeeudxkj234wpgbxxezi1eb104,UserGamer51 _,2015-05-27T17:33:07.626000,eminem is a ginius stop!﻿,0
346,z132enrpoy35yxpoe04cjr4zur3jvbyq3xo0k,Kasia Fabisiewicz,2015-06-05T19:02:05,&lt;3﻿,0
336,z13dxxabcp3ggby5y04cilbz0ojlyprwt1g,‫مريم الهندي‬‎,2015-01-10T22:00:57.333000,Check out this video on YouTube:﻿,1
86,z13oexlyfxivznriw04cdxmqqpmgy3bi5z40k,Jimmy Hopkins,2015-05-27T20:02:52.297000,this fucking song like a&#39;n oreo the only w...,0
334,z13szfnqoxaouldyf04cgrp5tmichhawj2g,FudgePlaysMC,2014-11-10T18:18:08,My 6th grade teacher looked exactly like Katy ...,0
220,LneaDw26bFtZQLtaItMp2bSQS4mrfYeKiGUTvOg1jrU,Louis Bryant,NaN,You guys should check out this EXTRAORDINARY w...,1
175,LneaDw26bFuRc-oJgCqkRGDhRb4gOA2s1-Katw83fJI,Derek Moya,NaN,You guys should check out this EXTRAORDINARY w...,1
304,z12xjtzzfw2rtdt5p04cfbmihmusv3hjouc0k,Yourmomma Son,2014-11-06T22:17:27,This is so stupid. If you Roared at a Lion in ...,0
196,z13ptttyrxekcdvib04chdepnmqygrnwj4c,Domonic gamer,2015-05-24T23:53:16.454000,is it bad that my realtionship is just like th...,0
331,z13twpph2vq5zv15k04cixdrfubnxh3oqz40k,Seth Ryan,2015-05-21T20:04:42.844000,Amazing﻿,0


In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words, max_features=4328)
Y = dataset["CLASS"]
wnl = WordNetLemmatizer()
stemmer = PorterStemmer()

In [42]:
# X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(wnl.lemmatize))
X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(stemmer.stem))

In [64]:
# model = svm.SVC(kernel='poly', C=1, degree=2)
model = LogisticRegression(C=1000, solver='lbfgs', penalty='l2', max_iter=10000)

In [65]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_lem, Y, test_size=0.2, random_state=42)

In [66]:
model.fit(X_train1, y_train1)

LogisticRegression(C=1000, max_iter=10000)

In [67]:
from pickle import dump
with open("resources/models/all/spam-filtering-lgr-tubespam.pkl", "wb") as f:
    dump(model, f, protocol=5)

In [49]:
from sklearn.metrics import *
 
accuracy_score(y_test1, model.predict(X_test1)), precision_score(y_test1, model.predict(X_test1))

(0.9362244897959183, 0.961352657004831)

In [50]:
path_to_posts = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/comments"

filenames = os.listdir(path_to_posts)
comments = [pd.read_csv(path_to_posts + '/' + filename, sep='\t') for filename in filenames]

comments = pd.concat(comments)

In [51]:
comments.head(10)

,TEAM_ABBREVIATION,COMMENT
0,PHX,This is Grayson Allen’s team. Kd and Book are ...
1,PHX,Donjoe Lampe KD only wins when he is with star...
2,PHX,Grayson carried for 3quarters and KD brought i...
3,NaN,Michelle Witman no
4,PHX,Grayson Allen's shoulders must hurt for carryi...
5,PHX,Grayson better get a spot in the 3P contest. 🔥
6,PHX,3rd straight wins for the #suns!!! what a run ...
7,PHX,It’s time to start considering Durant a greate...
8,PHX,"Well done, KD!"
9,NaN,Chris Paynter hello


In [52]:
comments["SPAM"] = comments["TEAM_ABBREVIATION"].apply(lambda x: 1 if x == "SPAM" else 0)

In [53]:
comments[comments['SPAM'] == 0].describe()

,SPAM
count,24762.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [54]:
test_comments = comments.sample(frac=0.5, random_state=1)

In [55]:
test_comments[test_comments["SPAM"] == 1].describe()

,SPAM
count,1109.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [56]:
X_comments_lem = vectorizer.fit_transform(test_comments["COMMENT"].apply(wnl.lemmatize))
Y_comments = test_comments["SPAM"]

In [57]:
from sklearn.metrics import *

y_pred = model.predict(X_comments_lem)

In [58]:
accuracy_score(Y_comments, y_pred), precision_score(Y_comments, y_pred)

(0.9062801932367149, 0.38414634146341464)

In [35]:
test_comments[test_comments["SPAM"] == y_pred].describe()


,SPAM
count,12271.000000
mean,0.000815
std,0.028536
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [68]:
from sklearn.model_selection import train_test_split

vectorizer2= CountVectorizer(stop_words=stop_words, max_features=4328)
model2 = LogisticRegression(C=1, solver='newton-cg', penalty='l2', max_iter=10000)
X_lem2 = vectorizer2.fit_transform(comments["COMMENT"].apply(wnl.lemmatize))
# X_stem2 = vectorizer2.fit_transform(test_comments["COMMENT"].apply(stemmer.stem))
Y2 = comments["SPAM"]

X_train, X_test, y_train, y_test = train_test_split(X_lem2, Y2, test_size=0.33, random_state=42)
model2.fit(X_train, y_train)




LogisticRegression(C=1, max_iter=10000, solver='newton-cg')

In [69]:
y_pred = model2.predict(X_test)
accuracy_score(y_test, y_pred), precision_score(y_test, y_pred)

(0.9782682130390722, 0.9057437407952872)

In [70]:
from pickle import dump
with open("resources/models/all/spam-filtering-lgr-nba.pkl", "wb") as f:
    dump(model2, f, protocol=5)

In [66]:
from sklearn.model_selection import KFold, GridSearchCV


def hypertune_svm(x, y):
    
    lr = LogisticRegression()
    penalty1 = ['l1']
    solvers1 = ['saga', 'liblinear']
    c_values = [100,50, 10, 1.0, 0.1, 0.01]
    lgr_grid1 = dict(solver=solvers1, penalty=penalty1, C=c_values)
    
    penalty2 = ['l2']
    solvers2 = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    lgr_grid2 = dict(solver=solvers2, penalty=penalty2, C=c_values)
    
    
    
    # model = svm.SVC()
    # kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    # C = [1000, 100, 50, 10, 1.0, 0.1, 0.01]
    # gamma = ['scale']
    # degree = range(1,11)
    # # define grid search
    # grid = dict(kernel=kernel,C=C,gamma=gamma, degree=degree)

    cv = KFold(n_splits=10)
    grid_search = GridSearchCV(estimator=lr, param_grid=lgr_grid1, n_jobs=-1, cv=cv, scoring='precision', error_score=0, verbose=2)
    grid_result = grid_search.fit(x, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [67]:
hypertune_svm(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits



KeyboardInterrupt



In [ ]:
# model2= svm.SVC(kernel='poly', C=1, degree=2)

model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred), precision_score(y_test, y_pred)